# Testing model building with `carveme` and `memote`

Importing the COBRA module to read sbml models created via the `carveme` method. Subsequently we reconstruct a draft of the GSM model of *S. cerevisae* using the found RefSeq ID. Looking at the [reference genome R64](https://www.ncbi.nlm.nih.gov/data-hub/taxonomy/4932/?utm_source=assembly&utm_medium=referral&utm_campaign=KnownItemSensor:taxname), we see that 6464 have been annotated.  

In [1]:
from cobra.io import read_sbml_model

%%capture
!carve --refseq GCF_000146045.2_R64 --output Saccharomyces.xml

print("DONE WITH THE CARVING!")

In [2]:
model = read_sbml_model('yeast-GEM.xml') # Model initiated in minimal medium + glucose (flux bound = 1.0)

In [3]:
print("GSM model fact sheet:")
print("Number of reactions:", len(model.reactions))
print("Number of metabolites:", len(model.metabolites))
print("Number of genes included in the model:", len(model.genes))
print("% of genes in model compared to the number of coding genes in Saccharomyces cerevisae R64 (6464 genes):",
      round((len(model.genes)/6464)*100, 2))
print("Gene coverage percentage =", round(len(model.genes)/len(model.reactions)*100, 2), "%")

GSM model fact sheet:
Number of reactions: 4058
Number of metabolites: 2742
Number of genes included in the model: 1150
% of genes in model compared to the number of coding genes in Saccharomyces cerevisae R64 (6464 genes): 17.79
Gene coverage percentage = 28.34 %


Checking the model setup - how the metabolites etc. are named/tagged

In [4]:
model.metabolites[len(model.metabolites)-10:len(model.metabolites)-1]

[<Metabolite s_4252[p] at 0x7fb896016430>,
 <Metabolite s_4253[m] at 0x7fb896016460>,
 <Metabolite s_4254[c] at 0x7fb896016490>,
 <Metabolite s_4255[e] at 0x7fb8960164c0>,
 <Metabolite s_4256[c] at 0x7fb8960164f0>,
 <Metabolite s_4257[e] at 0x7fb8960160a0>,
 <Metabolite s_4258[c] at 0x7fb896016550>,
 <Metabolite s_4259[e] at 0x7fb896016520>,
 <Metabolite s_4261[c] at 0x7fb896016700>]

In [5]:
model.reactions[len(model.reactions)-10:len(model.reactions)]

[<Reaction r_4689 at 0x7fb87dedadc0>,
 <Reaction r_4690 at 0x7fb87df464c0>,
 <Reaction r_4691 at 0x7fb87dedae50>,
 <Reaction r_4692 at 0x7fb87deda5b0>,
 <Reaction r_4693 at 0x7fb87deda580>,
 <Reaction r_4694 at 0x7fb87deefbb0>,
 <Reaction r_4695 at 0x7fb87deefcd0>,
 <Reaction r_4697 at 0x7fb87deef340>,
 <Reaction r_4698 at 0x7fb87deefe20>,
 <Reaction r_4699 at 0x7fb87deefd60>]

In [6]:
model.compartments # compartments used

{'ce': 'cell envelope',
 'c': 'cytoplasm',
 'e': 'extracellular',
 'm': 'mitochondrion',
 'n': 'nucleus',
 'p': 'peroxisome',
 'er': 'endoplasmic reticulum',
 'g': 'Golgi',
 'lp': 'lipid particle',
 'v': 'vacuole',
 'erm': 'endoplasmic reticulum membrane',
 'vm': 'vacuolar membrane',
 'gm': 'Golgi membrane',
 'mm': 'mitochondrial membrane'}

In [7]:
model.metabolites.get_by_id('s_0001[ce]') # Again specifying model naming

Metabolite identifier,s_0001[ce]
Name,(1->3)-beta-D-glucan [cell envelope]
Memory address,0x07fb87e3179a0
Formula,C6H10O5
Compartment,ce
In 3 reaction(s),"r_0005, r_1543, r_4048"


In [8]:
# Investigating where in the model, acetyl-CoA is used
for metabolite in model.metabolites.query('acetyl-CoA', 'name'):
    print(metabolite.name)

acetoacetyl-CoA [cytoplasm]
acetoacetyl-CoA [mitochondrion]
acetyl-CoA [cytoplasm]
acetyl-CoA [mitochondrion]
acetyl-CoA [nucleus]
acetyl-CoA [peroxisome]
acetoacetyl-CoA [peroxisome]


In [9]:
# Model default objective function (growth rate)
print("Model objective function:\n", model.objective, sep="")

Model objective function:
Maximize
1.0*r_2111 - 1.0*r_2111_reverse_58b69


In [10]:
# Investigating the growth reaction rate
model.reactions.get_by_id('r_2111')

Reaction identifier,r_2111
Name,growth
Memory address,0x07fb894db7a30
Stoichiometry,s_0450[c] --> biomass [cytoplasm] -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
model.metabolites.get_by_id('s_0450[c]')

Metabolite identifier,s_0450[c]
Name,biomass [cytoplasm]
Memory address,0x07fb8963bab50
Formula,
Compartment,c
In 2 reaction(s),"r_2111, r_4041"


In [12]:
model.optimize()

,fluxes,reduced_costs
r_0001,0.000000,-1.667286e-02
r_0002,0.000000,-1.667286e-02
r_0003,0.000000,-0.000000e+00
r_0004,0.000000,1.734723e-18
r_0005,0.062686,6.722053e-18
...,...,...
r_4694,0.000000,0.000000e+00
r_4695,0.000000,0.000000e+00
r_4697,0.000000,-0.000000e+00
r_4698,0.000000,-1.866638e-01


In [13]:
# Exchange reactions for uptake of nutrients from medium
for r in model.medium.keys():
    print(model.reactions.get_by_id(r).name, "=",model.medium[r])

ammonium exchange = 1000.0
D-glucose exchange = 1.0
H+ exchange = 1000.0
iron(2+) exchange = 1000.0
oxygen exchange = 1000.0
phosphate exchange = 1000.0
potassium exchange = 1000.0
sodium exchange = 1000.0
sulphate exchange = 1000.0
water exchange = 1000.0
chloride exchange = 1000.0
Cu2(+) exchange = 1000.0
Mn(2+) exchange = 1000.0
Zn(2+) exchange = 1000.0
Mg(2+) exchange = 1000.0
Ca(2+) exchange = 1000.0


In [14]:
model.medium

{'r_1654': 1000.0,
 'r_1714': 1.0,
 'r_1832': 1000.0,
 'r_1861': 1000.0,
 'r_1992': 1000.0,
 'r_2005': 1000.0,
 'r_2020': 1000.0,
 'r_2049': 1000.0,
 'r_2060': 1000.0,
 'r_2100': 1000.0,
 'r_4593': 1000.0,
 'r_4594': 1000.0,
 'r_4595': 1000.0,
 'r_4596': 1000.0,
 'r_4597': 1000.0,
 'r_4600': 1000.0}

In [15]:
model.reactions.r_1714

Reaction identifier,r_1714
Name,D-glucose exchange
Memory address,0x07fb894f10f40
Stoichiometry,s_0565[e] <=> D-glucose [extracellular] <=>
GPR,
Lower bound,-1.0
Upper bound,1000.0


In [16]:
model.metabolites.get_by_id('s_0565[e]')

Metabolite identifier,s_0565[e]
Name,D-glucose [extracellular]
Memory address,0x07fb8963ded90
Formula,C6H12O6
Compartment,e
In 6 reaction(s),"r_4400, r_1024, r_0370, r_1714, r_1166, r_4420"


In [17]:
# Experimentation with glucose concentration --> reach maximum growth rate at conc. ca. 575 
medium = model.medium
min_gluc = model.optimize()
print("mu_min_gluc =", min_gluc.fluxes['r_2111'])
with model:
    medium['r_1714'] = 575  # Glucose exchange reaciton
    model.medium = medium
    solution = model.optimize()
    print("mu_max_gluc =",solution.fluxes['r_2111'])

mu_min_gluc = 0.08374778664999834
mu_max_gluc = 19.816830259464282


In [18]:
model.medium

{'r_1654': 1000.0,
 'r_1714': 1.0,
 'r_1832': 1000.0,
 'r_1861': 1000.0,
 'r_1992': 1000.0,
 'r_2005': 1000.0,
 'r_2020': 1000.0,
 'r_2049': 1000.0,
 'r_2060': 1000.0,
 'r_2100': 1000.0,
 'r_4593': 1000.0,
 'r_4594': 1000.0,
 'r_4595': 1000.0,
 'r_4596': 1000.0,
 'r_4597': 1000.0,
 'r_4600': 1000.0}

In [19]:
for reaction in model.metabolites.get_by_id('s_0367[c]').reactions:
    reaction
    #print(reaction.name, reaction, sep = " | ")

In [20]:
model.metabolites.get_by_id('s_0367[c]').reactions

frozenset({<Reaction r_0103 at 0x7fb895d8ab50>,
           <Reaction r_0559 at 0x7fb895acdf40>,
           <Reaction r_4396 at 0x7fb87e054f70>})

In [63]:
#Does IPP, isopentenyl pyrophosphate, exist in our model?
for metabolite in model.metabolites.query("diphosphate", 'name'):
    print(metabolite.name)
    print(metabolite.id)

(2-amino-4-hydroxy-7,8-dihydropteridin-6-yl)methyl trihydrogen diphosphate [mitochondrion]
s_0006[m]
geranylgeranyl diphosphate [cytoplasm]
s_0189[c]
farnesyl diphosphate [cytoplasm]
s_0190[c]
farnesyl diphosphate [mitochondrion]
s_0191[m]
decaprenyl diphosphate [lipid particle]
s_0606[lp]
diphosphate [cytoplasm]
s_0633[c]
diphosphate [lipid particle]
s_0635[lp]
diphosphate [mitochondrion]
s_0636[m]
diphosphate [nucleus]
s_0637[n]
diphosphate [peroxisome]
s_0638[p]
docosaprenyl diphosphate [lipid particle]
s_0640[lp]
dodecaprenyl diphosphate [lipid particle]
s_0641[lp]
farnesyl diphosphate [lipid particle]
s_0692[lp]
geranyl diphosphate [cytoplasm]
s_0745[c]
geranylgeranyl diphosphate [lipid particle]
s_0746[lp]
henicosaprenyl diphosphate [lipid particle]
s_0813[lp]
heptadecaprenyl diphosphate [lipid particle]
s_0814[lp]
heptaprenyl diphosphate [lipid particle]
s_0815[lp]
hexadecaprenyl diphosphate [lipid particle]
s_0829[lp]
hexaprenyl diphosphate [lipid particle]
s_0830[lp]
hexapreny

In [22]:
#Checking whether IPP, isopentenyl pyrophosphate, can be found in the model. It should be produced from mevalonate
for metabolite in model.reactions.query('meva',"name"):
    print(metabolite.name)
    s=""
    for r in range(len(metabolite.reactants)):
        if r==len(metabolite.reactants)-1:
            s+=metabolite.reactants[r].name+" ==> "
        else:
            s+=metabolite.reactants[r].name+" + "
    for r in range(len(metabolite.products)):
        if r==len(metabolite.products)-1:
            s+=metabolite.products[r].name
        else:
            s+=metabolite.products[r].name+" + "
    print(s)

mevalonate kinase (atp)
(R)-mevalonate [cytoplasm] + ATP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + ADP [cytoplasm] + H+ [cytoplasm]
mevalonate kinase (ctp)
(R)-mevalonate [cytoplasm] + CTP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + CDP [cytoplasm] + H+ [cytoplasm]
mevalonate kinase (gtp)
(R)-mevalonate [cytoplasm] + GTP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + GDP [cytoplasm] + H+ [cytoplasm]
mevalonate kinase (UTP)
(R)-mevalonate [cytoplasm] + UTP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + H+ [cytoplasm] + UDP [cytoplasm]
mevalonate pyrophoshate decarboxylase
(R)-5-diphosphomevalonic acid [cytoplasm] + ATP [cytoplasm] ==> ADP [cytoplasm] + carbon dioxide [cytoplasm] + isopentenyl diphosphate [cytoplasm] + phosphate [cytoplasm]
phosphomevalonate kinase
(R)-5-phosphomevalonic acid [cytoplasm] + ATP [cytoplasm] ==> (R)-5-diphosphomevalonic acid [cytoplasm] + ADP [cytoplasm]
(R)-mevalonate transport
(R)-mevalonate [extracellul

From the above tests it can be seen that is in the model (5-posphomevalonic acid) 

In [23]:
#What reactions does mevalonate participate in
for metabolite in model.reactions:
    for met in metabolite.reactants and metabolite.products:
        if "meval" in met.name:
            print(metabolite.name)
            s=""
            for r in range(len(metabolite.reactants)):
                if r==len(metabolite.reactants)-1:
                    s+=metabolite.reactants[r].name+" ==> "
                else:
                    s+=metabolite.reactants[r].name+" + "
            for r in range(len(metabolite.products)):
                if r==len(metabolite.products)-1:
                    s+=metabolite.products[r].name
                else:
                    s+=metabolite.products[r].name+" + "
            print(s)

hydroxymethylglutaryl CoA reductase
3-hydroxy-3-methylglutaryl-CoA [cytoplasm] + H+ [cytoplasm] + NADPH [cytoplasm] ==> (R)-mevalonate [cytoplasm] + coenzyme A [cytoplasm] + NADP(+) [cytoplasm]
mevalonate kinase (atp)
(R)-mevalonate [cytoplasm] + ATP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + ADP [cytoplasm] + H+ [cytoplasm]
mevalonate kinase (ctp)
(R)-mevalonate [cytoplasm] + CTP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + CDP [cytoplasm] + H+ [cytoplasm]
mevalonate kinase (gtp)
(R)-mevalonate [cytoplasm] + GTP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + GDP [cytoplasm] + H+ [cytoplasm]
mevalonate kinase (UTP)
(R)-mevalonate [cytoplasm] + UTP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + H+ [cytoplasm] + UDP [cytoplasm]
phosphomevalonate kinase
(R)-5-phosphomevalonic acid [cytoplasm] + ATP [cytoplasm] ==> (R)-5-diphosphomevalonic acid [cytoplasm] + ADP [cytoplasm]
(R)-mevalonate transport
(R)-mevalonate [extracellular] ==> (R)-mev

In [24]:
#What reactions does IPP or 5-phosphomevalonic acid participate in
for metabolite in model.reactions:
    for met in metabolite.reactants and metabolite.products:
        if "phosphomevalon" in met.name:
            print(metabolite)
            s=""
            for r in range(len(metabolite.reactants)):
                if r==len(metabolite.reactants)-1:
                    s+=metabolite.reactants[r].name+" ==> "
                else:
                    s+=metabolite.reactants[r].name+" + "
            for r in range(len(metabolite.products)):
                if r==len(metabolite.products)-1:
                    s+=metabolite.products[r].name
                else:
                    s+=metabolite.products[r].name+" + "
            print(s)

r_0735: s_0028[c] + s_0434[c] --> s_0019[c] + s_0394[c] + s_0794[c]
(R)-mevalonate [cytoplasm] + ATP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + ADP [cytoplasm] + H+ [cytoplasm]
r_0736: s_0028[c] + s_0539[c] --> s_0019[c] + s_0467[c] + s_0794[c]
(R)-mevalonate [cytoplasm] + CTP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + CDP [cytoplasm] + H+ [cytoplasm]
r_0737: s_0028[c] + s_0785[c] --> s_0019[c] + s_0739[c] + s_0794[c]
(R)-mevalonate [cytoplasm] + GTP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + GDP [cytoplasm] + H+ [cytoplasm]
r_0738: s_0028[c] + s_1559[c] --> s_0019[c] + s_0794[c] + s_1538[c]
(R)-mevalonate [cytoplasm] + UTP [cytoplasm] ==> (R)-5-phosphomevalonic acid [cytoplasm] + H+ [cytoplasm] + UDP [cytoplasm]
r_0904: s_0019[c] + s_0434[c] --> s_0018[c] + s_0394[c]
(R)-5-phosphomevalonic acid [cytoplasm] + ATP [cytoplasm] ==> (R)-5-diphosphomevalonic acid [cytoplasm] + ADP [cytoplasm]


In [25]:
#What reactions does IPP or 5-diphosphomevalonic acid participate in
for metabolite in model.reactions:
    for met in metabolite.reactants and metabolite.products:
        if "diphosphomevalon" in met.name:
            print(metabolite)
            s=""
            for r in range(len(metabolite.reactants)):
                if r==len(metabolite.reactants)-1:
                    s+=metabolite.reactants[r].name+" ==> "
                else:
                    s+=metabolite.reactants[r].name+" + "
            for r in range(len(metabolite.products)):
                if r==len(metabolite.products)-1:
                    s+=metabolite.products[r].name
                else:
                    s+=metabolite.products[r].name+" + "
            print(s)

r_0904: s_0019[c] + s_0434[c] --> s_0018[c] + s_0394[c]
(R)-5-phosphomevalonic acid [cytoplasm] + ATP [cytoplasm] ==> (R)-5-diphosphomevalonic acid [cytoplasm] + ADP [cytoplasm]


It seems that it ends with 5-diphosphomevalonic acid which means that the rest of the pathway needs to be introduced as expected

In [62]:
#What is the id of 5-diphosphomevalonic acid?
for i in model.metabolites.query("isopentenyl diphosphate", 'name'):
    for metabolite in i.reactions:
        for met in metabolite.reactants and metabolite.products:
            if True:
                print(metabolite)
                s=""
                for r in range(len(metabolite.reactants)):
                    if r==len(metabolite.reactants)-1:
                        s+=metabolite.reactants[r].name+" ==> "
                    else:
                        s+=metabolite.reactants[r].name+" + "
                for r in range(len(metabolite.products)):
                    if r==len(metabolite.products)-1:
                        s+=metabolite.products[r].name
                    else:
                        s+=metabolite.products[r].name+" + "
                print(s+"\n")

r_0373: s_0190[c] + s_0943[c] --> s_0189[c] + s_0633[c]
farnesyl diphosphate [cytoplasm] + isopentenyl diphosphate [cytoplasm] ==> geranylgeranyl diphosphate [cytoplasm] + diphosphate [cytoplasm]

r_0373: s_0190[c] + s_0943[c] --> s_0189[c] + s_0633[c]
farnesyl diphosphate [cytoplasm] + isopentenyl diphosphate [cytoplasm] ==> geranylgeranyl diphosphate [cytoplasm] + diphosphate [cytoplasm]

r_0462: s_0745[c] + s_0943[c] --> s_0190[c] + s_0633[c]
geranyl diphosphate [cytoplasm] + isopentenyl diphosphate [cytoplasm] ==> farnesyl diphosphate [cytoplasm] + diphosphate [cytoplasm]

r_0462: s_0745[c] + s_0943[c] --> s_0190[c] + s_0633[c]
geranyl diphosphate [cytoplasm] + isopentenyl diphosphate [cytoplasm] ==> farnesyl diphosphate [cytoplasm] + diphosphate [cytoplasm]

r_0355: s_0943[c] + s_1376[c] --> s_0633[c] + s_0745[c]
isopentenyl diphosphate [cytoplasm] + prenyl diphosphate(3-) [cytoplasm] ==> diphosphate [cytoplasm] + geranyl diphosphate [cytoplasm]

r_0355: s_0943[c] + s_1376[c] --> 

In [27]:
model.metabolites.get_by_id("s_1322[c]")

Metabolite identifier,s_1322[c]
Name,phosphate [cytoplasm]
Memory address,0x07fb8962c9fd0
Formula,HO4P
Compartment,c
In 188 reaction(s),"r_4567, r_4220, r_2228, r_0073, r_3187, r_0222, r_2219, r_3649, r_1620, r_4216, r_0013, r_2226, r_0032, r_2220, r_3663, r_4205, r_4435, r_4607, r_0949, r_1603, r_4357, r_0109, r_4376, r_0227,..."


In [28]:
#We need to add the necessary reactions
from cobra import Model, Reaction, Metabolite
r1=Reaction("r_4700")
r1.name = 'Diphosphomevalonate decarboxylation'
Diphos=model.metabolites.get_by_id("s_0018[c]")
#https://en.wikipedia.org/wiki/Phosphomevalonic_acid
Iso_di = Metabolite(
    's_4262[c]', 
    formula='C5H9O7P2', 
    name='Isopentenyl diphosphate [cytoplasm]', 
    compartment='c')
ATP=model.metabolites.get_by_id("s_0434[c]")
ADP=model.metabolites.get_by_id("s_0394[c]")
P=model.metabolites.get_by_id("s_1322[c]")
CO2=model.metabolites.get_by_id("s_0456[c]")
#https://en.wikipedia.org/wiki/Diphosphomevalonate_decarboxylase
r1.add_metabolites({Diphos:-1.0,ATP:-1.0,ADP:1.0,Iso_di:1.0,P:1.0,CO2:1.0})
r1.reaction
r1.gene_reaction_rule="(STM000001)"

In [ ]:
r2=Reaction("r_4701")
r2.name=""

In [29]:
model.add_reactions([r1])
model.reactions.get_by_id("r_4700")

Reaction identifier,r_4700
Name,Diphosphomevalonate decarboxylation
Memory address,0x07fb87db373a0
Stoichiometry,s_0018[c] + s_0434[c] --> s_0394[c] + s_0456[c] + s_1322[c] + s_4262[c] (R)-5-diphosphomevalonic acid [cytoplasm] + ATP [cytoplasm] --> ADP [cytoplasm] + carbon dioxide [cytoplasm] + phosphate [cytoplasm] + Isopentenyl diphosphate [cytoplasm]
GPR,(STM000001)
Lower bound,0.0
Upper bound,1000.0


In [47]:
model.metabolites.get_by_id("s_0458[e]")

Metabolite identifier,s_0458[e]
Name,carbon dioxide [extracellular]
Memory address,0x07fb8963bacd0
Formula,CO2
Compartment,e
In 3 reaction(s),"r_1668, r_1672, r_1697"


In [42]:
solution = model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420[e],r_1654,0.5988,0,0.00%
s_0565[e],r_1714,1,6,100.00%
s_0796[e],r_1832,0.04236,0,0.00%
s_0925[e],r_1861,2.63E-06,0,0.00%
s_1277[e],r_1992,2.25,0,0.00%
s_1324[e],r_2005,0.545,0,0.00%
s_1374[e],r_2020,0.000304,0,0.00%
s_1438[e],r_2049,0.0003325,0,0.00%
s_1468[e],r_2060,0.007203,0,0.00%
s_4200[e],r_4593,0.000108,0,0.00%


In [45]:
with model:
    model.objective = model.reactions.r_4700
    print(model.objective)
    print(model.optimize().objective_value)

Maximize
1.0*r_4700 - 1.0*r_4700_reverse_15972
0.0


In [33]:
for r in solution.fluxes[solution.fluxes!=0].keys():
    print(model.reactions.get_by_id(r).name)

1,3-beta-glucan synthase
1,6-beta-glucan synthase
1-(5-phosphoribosyl)-5-[(5-phosphoribosylamino)methylideneamino)imidazole-4-carboxamide isomerase
2,3-diketo-5-methylthio-1-phosphopentane degradation reaction
2,5-diamino-6-ribitylamino-4(3H)-pyrimidinone 5'-phosphate deaminase
2,5-diamino-6-ribosylamino-4(3H)-pyrimidinone 5'-phosphate reductase (NADPH)
2-aceto-2-hydroxybutanoate synthase
2-aminoadipate transaminase
2-dehydropantoate 2-reductase
2-deoxy-D-arabino-heptulosonate 7-phosphate synthetase
2-isopropylmalate hydratase
2-isopropylmalate synthase
2-keto-4-methylthiobutyrate transamination
2-methylcitrate dehydratase
2-oxo-4-methyl-3-carboxypentanoate decarboxylation
3',5'-bisphosphate nucleotidase
3,4-dihydroxy-2-butanone-4-phosphate synthase
3-dehydroquinate dehydratase
3-dehydroquinate synthase
3-hydroxy-L-kynurenine hydrolase
3-hydroxyanthranilate 3,4-dioxygenase
3-isopropylmalate dehydratase
3-isopropylmalate dehydrogenase
3-methyl-2-oxobutanoate hydroxymethyltransferase
3-p